In [9]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

# Local imports
import import_ipynb
import model

In [10]:
def solve(robot: model.RobotIface, ode_system, control, state_from:np.array, time_from=0, time_to=20) -> (np.array, np.matrix, np.matrix, np.matrix):
    time_grid = np.linspace(time_from, time_to, 1000)
    ode_solver = integrate.ode(ode_system(control))
    ode_solver.set_integrator('vode', method='bdf', order=15, nsteps=3000)
    ode_solver.set_initial_value(state_from, time_from)

    end_effectors = np.zeros((len(time_grid), 3)); controls = np.zeros_like(end_effectors)
    params = np.zeros((len(time_grid), state_from.shape[0] - 6))
    for i in range(len(time_grid)):
        state = state_from
        if i>0:
            state = ode_solver.integrate(time_grid[i])
        end_effectors[i, :] = np.squeeze(robot.forward_kinematics(state))
        controls[i, :] = np.squeeze(robot.jacobian(state).T @ control(state, time_grid[i]))
        params[i, :] = state[6:]
    return time_grid, end_effectors, controls, params

In [11]:
def plot_3d(grid:np.array, mesh:np.matrix, grid_name='t', mesh_name='x'):
    figure = plt.figure(figsize=(20, 20))
    
    one   = figure.add_subplot(2, 2, 1); one.set(title="$"+mesh_name+"_1$", xlabel=grid_name);   one.plot(grid, mesh[:, 0]);   one.grid('on')
    two   = figure.add_subplot(2, 2, 2); two.set(title="$"+mesh_name+"_2$", xlabel=grid_name);   two.plot(grid, mesh[:, 1]);   two.grid('on')
    three = figure.add_subplot(2, 2, 3); three.set(title="$"+mesh_name+"_3$", xlabel=grid_name); three.plot(grid, mesh[:, 2]); three.grid('on')
    four  = figure.add_subplot(2, 2, 4, projection='3d')
    four.set(title = 'Trajectory', xlabel="$"+mesh_name+"_1$", ylabel="$"+mesh_name+"_2$", zlabel="$"+mesh_name+"_3$")
    four.plot(mesh[:, 0], mesh[:, 1], mesh[:, 2], linewidth=2)
    
    return (figure, (one, two, three, four))